In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

from data_processing import *

from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.metrics import multilabel_confusion_matrix, balanced_accuracy_score
from sklearn.utils import class_weight

from keras.models import Sequential, load_model
from keras.layers import Embedding, Bidirectional, GRU, Dense, Dropout, BatchNormalization, LSTM, InputLayer, Flatten
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MathildeElimas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MathildeElimas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MathildeElimas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Using TensorFlow backend.


In [2]:
PATH = os.getcwd().replace('\\models', '') + '\\data'
method = 'word2vec'

In [3]:
intent, unique_intent, sentences = load_data(PATH + '\\intent_dialogues.csv')

                                            Sentence           Intent
0  Bonjour j'ai rendu mon équipement box est je n...  probleme_box_tv
1                                            Bonjour  probleme_box_tv
2                              Box internet en panne  probleme_box_tv
3  Plus de phone ni internet depuis plus ou moins...  probleme_box_tv
4  Bonjour j'ai la télécommande de ma box qui ne ...  probleme_box_tv


#### Text embedding

In [4]:
idx, cleaned_words = clean_sentences(sentences)
intent = list(map(intent.__getitem__, idx))
max_length = max(len(x.split()) for x in cleaned_words)

2020-10-30 18:48:16,733 - spacy_lefff.lefff - INFO - New LefffLemmatizer instantiated.
2020-10-30 18:48:16,733 - spacy_lefff.lefff - INFO - Reading lefff data...
2020-10-30 18:48:17,724 - spacy_lefff.lefff - INFO - Successfully loaded lefff lemmatizer


In [5]:
vocab = create_vocabulary(cleaned_words)

2020-10-30 18:49:24,513 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [6]:
encoded_doc = text_embedding(cleaned_words, vocab = list(vocab.keys()), encoding = method, max_length = max_length)
encoded_doc.shape

8
(26, 300)
3
(31, 300)
4
(30, 300)
16
(18, 300)
3
(31, 300)
5
(29, 300)
3
(31, 300)
3
(31, 300)
2
(32, 300)
11
(23, 300)
8
(26, 300)
3
(31, 300)
6
(28, 300)
2
(32, 300)
2
(32, 300)
2
(32, 300)
3
(31, 300)
2
(32, 300)
8
(26, 300)
3
(31, 300)
4
(30, 300)
18
(16, 300)
16
(18, 300)
5
(29, 300)
15
(19, 300)
4
(30, 300)
3
(31, 300)
2
(32, 300)
19
(15, 300)
5
(29, 300)
11
(23, 300)
4
(30, 300)
5
(29, 300)
4
(30, 300)
10
(24, 300)
2
(32, 300)
2
(32, 300)
7
(27, 300)
9
(25, 300)
3
(31, 300)
2
(32, 300)
2
(32, 300)
6
(28, 300)
3
(31, 300)
8
(26, 300)
6
(28, 300)
4
(30, 300)
3
(31, 300)
12
(22, 300)
3
(31, 300)
8
(26, 300)
2
(32, 300)
2
(32, 300)
3
(31, 300)
11
(23, 300)
7
(27, 300)
5
(29, 300)
16
(18, 300)
4
(30, 300)
22
(12, 300)
6
(28, 300)
6
(28, 300)
4
(30, 300)
5
(29, 300)
11
(23, 300)
7
(27, 300)
7
(27, 300)
7
(27, 300)
8
(26, 300)
14
(20, 300)
12
(22, 300)
15
(19, 300)
2
(32, 300)
2
(32, 300)
2
(32, 300)
4
(30, 300)
10
(24, 300)
5
(29, 300)
7
(27, 300)
3
(31, 300)
7
(27, 300)
3
(31, 300)

7
(27, 300)
21
(13, 300)
13
(21, 300)
5
(29, 300)
9
(25, 300)
5
(29, 300)
7
(27, 300)
3
(31, 300)
8
(26, 300)
2
(32, 300)
4
(30, 300)
10
(24, 300)
2
(32, 300)
3
(31, 300)
11
(23, 300)
10
(24, 300)
12
(22, 300)
4
(30, 300)
3
(31, 300)
9
(25, 300)
11
(23, 300)
5
(29, 300)
11
(23, 300)
21
(13, 300)
4
(30, 300)
17
(17, 300)
4
(30, 300)
3
(31, 300)
6
(28, 300)
2
(32, 300)
2
(32, 300)
7
(27, 300)
9
(25, 300)
2
(32, 300)
4
(30, 300)
8
(26, 300)
4
(30, 300)
2
(32, 300)
3
(31, 300)
9
(25, 300)
2
(32, 300)
18
(16, 300)
6
(28, 300)
2
(32, 300)
9
(25, 300)
3
(31, 300)
4
(30, 300)
5
(29, 300)
10
(24, 300)
6
(28, 300)
5
(29, 300)
22
(12, 300)
2
(32, 300)
2
(32, 300)
7
(27, 300)
4
(30, 300)
2
(32, 300)
2
(32, 300)
18
(16, 300)
4
(30, 300)
11
(23, 300)
3
(31, 300)
5
(29, 300)
5
(29, 300)
11
(23, 300)
2
(32, 300)
3
(31, 300)
4
(30, 300)
7
(27, 300)
6
(28, 300)
8
(26, 300)
2
(32, 300)
5
(29, 300)
9
(25, 300)
13
(21, 300)
6
(28, 300)
8
(26, 300)
3
(31, 300)
8
(26, 300)
3
(31, 300)
6
(28, 300)
4
(30, 300)

(30, 300)
28
(6, 300)
11
(23, 300)
3
(31, 300)
30
(4, 300)
14
(20, 300)
7
(27, 300)
2
(32, 300)
3
(31, 300)
2
(32, 300)
10
(24, 300)
10
(24, 300)
5
(29, 300)
2
(32, 300)
5
(29, 300)
2
(32, 300)
10
(24, 300)
3
(31, 300)
8
(26, 300)
23
(11, 300)
8
(26, 300)
2
(32, 300)
10
(24, 300)
2
(32, 300)
9
(25, 300)
6
(28, 300)
5
(29, 300)
8
(26, 300)
5
(29, 300)
6
(28, 300)
3
(31, 300)
11
(23, 300)
14
(20, 300)
10
(24, 300)
6
(28, 300)
13
(21, 300)
5
(29, 300)
12
(22, 300)
2
(32, 300)
12
(22, 300)
7
(27, 300)
15
(19, 300)
3
(31, 300)
5
(29, 300)
12
(22, 300)
11
(23, 300)
5
(29, 300)
20
(14, 300)
4
(30, 300)
12
(22, 300)
7
(27, 300)
3
(31, 300)
7
(27, 300)
11
(23, 300)
5
(29, 300)
10
(24, 300)
12
(22, 300)
31
(3, 300)
2
(32, 300)
11
(23, 300)
8
(26, 300)
7
(27, 300)
2
(32, 300)
10
(24, 300)
19
(15, 300)
4
(30, 300)
26
(8, 300)
7
(27, 300)
2
(32, 300)
12
(22, 300)
20
(14, 300)
9
(25, 300)
6
(28, 300)
3
(31, 300)
10
(24, 300)
17
(17, 300)
5
(29, 300)
5
(29, 300)
9
(25, 300)
2
(32, 300)
10
(24, 300)
4

4
(30, 300)
6
(28, 300)
2
(32, 300)
2
(32, 300)
5
(29, 300)
13
(21, 300)
7
(27, 300)
21
(13, 300)
6
(28, 300)
9
(25, 300)
8
(26, 300)
3
(31, 300)
3
(31, 300)
3
(31, 300)
3
(31, 300)
6
(28, 300)
2
(32, 300)
8
(26, 300)
5
(29, 300)
15
(19, 300)
3
(31, 300)
8
(26, 300)
3
(31, 300)
2
(32, 300)
7
(27, 300)
13
(21, 300)
7
(27, 300)
9
(25, 300)
3
(31, 300)
2
(32, 300)
4
(30, 300)
19
(15, 300)
10
(24, 300)
3
(31, 300)
6
(28, 300)
6
(28, 300)
6
(28, 300)
13
(21, 300)
13
(21, 300)
3
(31, 300)
7
(27, 300)
3
(31, 300)
14
(20, 300)
10
(24, 300)
3
(31, 300)
4
(30, 300)
4
(30, 300)
9
(25, 300)
5
(29, 300)
5
(29, 300)
19
(15, 300)
4
(30, 300)
14
(20, 300)
17
(17, 300)
8
(26, 300)
9
(25, 300)
4
(30, 300)
3
(31, 300)
4
(30, 300)
4
(30, 300)
6
(28, 300)
5
(29, 300)
3
(31, 300)
4
(30, 300)
7
(27, 300)
6
(28, 300)
2
(32, 300)
3
(31, 300)
11
(23, 300)
4
(30, 300)
8
(26, 300)
18
(16, 300)
6
(28, 300)
4
(30, 300)
3
(31, 300)
3
(31, 300)
10
(24, 300)
4
(30, 300)
6
(28, 300)
4
(30, 300)
6
(28, 300)
11
(23, 300)

(2855, 34, 300)

#### Intent embedding

In [7]:
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')
output_tokenizer.word_index

{'probleme_box_tv': 1, 'reseau_mobile': 2, 'explication_facture': 3}

In [8]:
intent_encoder, encoded_output = intent_embedding(intent)
intents = intent_encoder.categories_

### Neural Network

#### Train/test

In [9]:
train_X, val_X, train_Y, val_Y = train_test_split(encoded_doc, encoded_output, test_size = 0.2, random_state = 123)

In [10]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (2284, 34, 300) and train_Y = (2284, 3)
Shape of val_X = (571, 34, 300) and val_Y = (571, 3)


#### Model definition and training

In [11]:
def create_model():
    model = Sequential()

    model.add(InputLayer(input_shape = encoded_doc.shape[1:]))
#     model.add(Embedding(vocab_size, 64))
#    model.add(Bidirectional(LSTM(128))) 
    model.add(Dense(64, activation = "relu"))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation = "relu"))
    model.add(Dropout(0.2))
#     model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(len(unique_intent), activation = "softmax")) 

    return model

In [12]:
model = create_model()

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["categorical_accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 34, 64)            19264     
_________________________________________________________________
dropout_1 (Dropout)          (None, 34, 64)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 34, 64)            4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 34, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2176)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 6531      
Total params: 29,955
Trainable params: 29,955
Non-trainable params: 0
__________________________________________________

In [13]:
labels = [np.argmax(x) for x in train_Y]
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(labels),
                                                 labels)

In [14]:
filename = os.getcwd().replace('\\models','') + '\\resultats\\NN.h5'
checkpoint = ModelCheckpoint(filename, monitor='categorical_accuracy', 
                             verbose=1, save_best_only=True, 
                             mode='max')

hist = model.fit(train_X, train_Y, #class_weight = class_weights,
                 epochs = 50, batch_size = 32, 
                 callbacks = [checkpoint])

Epoch 1/50
2284/2284 [==============================] - 1s 538us/step - loss: 0.9907 - categorical_accuracy: 0.5333

Epoch 00001: categorical_accuracy improved from -inf to 0.53327, saving model to C:\Users\MathildeElimas\OneDrive - Datatorii\Documents\BYTEL\code\intent_classification\resultats\NN.h5
Epoch 2/50
2284/2284 [==============================] - 1s 398us/step - loss: 0.7540 - categorical_accuracy: 0.6769

Epoch 00002: categorical_accuracy improved from 0.53327 to 0.67688, saving model to C:\Users\MathildeElimas\OneDrive - Datatorii\Documents\BYTEL\code\intent_classification\resultats\NN.h5
Epoch 3/50
2284/2284 [==============================] - 1s 400us/step - loss: 0.6773 - categorical_accuracy: 0.7202

Epoch 00003: categorical_accuracy improved from 0.67688 to 0.72023, saving model to C:\Users\MathildeElimas\OneDrive - Datatorii\Documents\BYTEL\code\intent_classification\resultats\NN.h5
Epoch 4/50
2284/2284 [==============================] - 1s 387us/step - loss: 0.6343 - c

2284/2284 [==============================] - 1s 379us/step - loss: 0.3047 - categorical_accuracy: 0.8849

Epoch 00034: categorical_accuracy did not improve from 0.88573
Epoch 35/50
2284/2284 [==============================] - 1s 373us/step - loss: 0.3071 - categorical_accuracy: 0.8800

Epoch 00035: categorical_accuracy did not improve from 0.88573
Epoch 36/50
2284/2284 [==============================] - 1s 375us/step - loss: 0.2827 - categorical_accuracy: 0.8927

Epoch 00036: categorical_accuracy improved from 0.88573 to 0.89273, saving model to C:\Users\MathildeElimas\OneDrive - Datatorii\Documents\BYTEL\code\intent_classification\resultats\NN.h5
Epoch 37/50
2284/2284 [==============================] - 1s 379us/step - loss: 0.2965 - categorical_accuracy: 0.8870

Epoch 00037: categorical_accuracy did not improve from 0.89273
Epoch 38/50
2284/2284 [==============================] - 1s 387us/step - loss: 0.2844 - categorical_accuracy: 0.8862

Epoch 00038: categorical_accuracy did not imp

#### Model evaluation

Test with test dataset

In [15]:
# model = load_model(filename)

In [16]:
pred_proba = model.predict(val_X)
pred_class = [np.argmax(x) for x in pred_proba]
val_Y_class = [np.argmax(x) for x in val_Y]
balanced_accuracy_score(val_Y_class, pred_class)

0.6989632766841186

Test with a sentence

In [17]:
def predictions(text, vocab = list(vocab.keys()), encoding = method, max_length = max_length):
    idx, clean = list(clean_sentences(text))
    encoded = text_embedding(clean, vocab, encoding, max_length)
    
    #Check for unknown words
    if [] in encoded:
        encoded = list(filter(None, encoded))

#     test_ls = np.array(test_ls).reshape(1, len(test_ls))

    pred = model.predict_proba(encoded)
    return pred

In [18]:
text = "Je ne comprends pas ma dernière facture. Il y a une augmentation de 2euros."
pred_text = predictions([text])
intent_encoder.inverse_transform(pred_text)

2020-10-30 18:51:07,705 - spacy_lefff.lefff - INFO - New LefffLemmatizer instantiated.
2020-10-30 18:51:07,706 - spacy_lefff.lefff - INFO - Token lefff_lemma already registered
2020-10-30 18:51:07,706 - spacy_lefff.lefff - INFO - Reading lefff data...
2020-10-30 18:51:08,718 - spacy_lefff.lefff - INFO - Successfully loaded lefff lemmatizer
5
(29, 300)


ValueError: could not broadcast input array from shape (34,300) into shape (5,300)